In [2]:
from src.melvin.stream import Stream
from src.melvin.Transcriber import Transcriber
from src.melvin.StreamTranscriber import StreamTranscriber

from src.eval.TimedStreamingTranscriber import TimedStreamingTranscriber
from src.eval.Dataset import Dataset
from src.eval.OutputHandler import OutputHandler

In [3]:
w = StreamTranscriber.for_gpu("large-v3-turbo", [0])
dataset = Dataset()

In [4]:
while True:
    id, X, y = next(dataset)
    if len(X)/32000 < 100:
        break
print(id)
print(len(X)/32000)

121-123859-0000
93.24


In [5]:
out = OutputHandler()
stream = Stream(w, 0, out)
transcriber = TimedStreamingTranscriber(stream, out)
y_pred = await transcriber.transcribe(X)

Transcribing audio data with 2983680 bytes


Transcribing: 933chunk [01:33,  9.97chunk/s]                      


In [6]:
out.final_words

[{'conf': 0.93457, 'start': 0.24, 'end': 0.82, 'word': 'You'},
 {'conf': 0.999512, 'start': 0.82, 'end': 1.14, 'word': 'are'},
 {'conf': 0.991211, 'start': 1.16, 'end': 1.46, 'word': 'my'},
 {'conf': 0.984863, 'start': 1.46, 'end': 1.82, 'word': 'all'},
 {'conf': 0.953125, 'start': 1.82, 'end': 1.98, 'word': 'the'},
 {'conf': 1.0, 'start': 1.98, 'end': 2.4, 'word': 'world'},
 {'conf': 0.368896, 'start': 2.4, 'end': 3.44, 'word': 'and'},
 {'conf': 0.808105, 'start': 3.44, 'end': 3.66, 'word': 'i'},
 {'conf': 0.999023, 'start': 3.66, 'end': 3.96, 'word': 'must'},
 {'conf': 1.0, 'start': 3.96, 'end': 4.48, 'word': 'strive'},
 {'conf': 0.998535, 'start': 4.48, 'end': 5.24, 'word': 'to'},
 {'conf': 0.999512, 'start': 5.24, 'end': 5.5, 'word': 'know'},
 {'conf': 0.999512, 'start': 5.5, 'end': 5.76, 'word': 'my'},
 {'conf': 0.998047, 'start': 5.76, 'end': 6.42, 'word': 'shames'},
 {'conf': 0.999512, 'start': 6.42, 'end': 6.74, 'word': 'and'},
 {'conf': 0.999756, 'start': 6.74, 'end': 7.26, 'w

In [7]:
for m in out.partial_predictions:
    print(f"{m['time']:.4f} {' '.join([w.word for w in m['result']])}")

1.6928  You  are
2.3874  You  are  my  old.
3.3643  you  are  my  all  the  world
4.3902  you  are  my  all  the  world  and  i  must
5.3853  you  are  my  all  the  world  and  i  must  strive
6.4063  You  are  my  all  the  world,  and  I  must  strive  to  know  my  shame.
7.4166  You  are  my  all  the  world,  and  I  must  strive  to  know  my  shames  and  praise.
8.4678  You  are  my  all  the  world,  and  I  must  strive  to  know  my  shames  and  praises  from  your  tongue.
9.4439  You  are  my  all  the  world,  and  I  must  strive  to  know  my  shames  and  praises  from  your  tongue.
10.4291  You  are  my  all  the  world,  and  I  must  strive  to  know  my  shames  and  praises  from  your  tongue.
11.4726  you  are  my  all  the  world  and  i  must  strive  to  know  my  shames  and  praises  from  your  tongue  none  else  to  me  nor
12.4594  you  are  my  all  the  world  and  i  must  strive  to  know  my  shames  and  praises  from  your  tongue  none  else 

In [ ]:
from pydub.utils import re

def __norm_word(word) -> str:
    text = word.lower()
    # Remove non-alphabetic characters using regular expression
    text = re.sub(r"[^a-z]", "", text)
    return text.lower().strip().strip(".,?!")

def remove_duplicates(words):
    i = 1
    new_words = words
    while i < len(new_words):
        if (
            __norm_word(new_words[i]["word"]) == __norm_word(new_words[i-1]["word"])
            and new_words[i]["start"] < new_words[i-1]["end"]
        ):
            new_words.pop(i)
        else:
            i += 1
    return new_words

In [ ]:
y_gold = []
for result in stream.final_transcriptions:
    y_gold += result["result"]

y_gold = remove_duplicates(y_gold)
y_gold_text = " ".join([w["word"] for w in y_gold])
print(y_gold_text)
y_gold

You are my all the world and i must strive to know my shames and praises from your tongue. none else to me nor i to none alive that my steeled sense or changes right or wrong. oh tis the first tis flattery in my seeing And my great mind most kingly drinks it up. mine eye well knows what with his gust is green and to his palate. doth prepare the cup, If it be poisoned, tis the lesser sin That mine eye loves it and doth first begin. But reckoning time, whose million'd accidents Creep in twixt vows, and change decrees of kings Tan's sacred beauty, blunt the sharpest intents, divert strong minds to the course of altering things. Alas, why fearing of time's then say, now I love you best. when i was certain or in certainty crowning the present Doubting of the rest? love is a babe Then might I not say so? to give full growth to that which still doth grow So I return rebuked to my content and gain by ill thrice more than


[{'conf': 0.93457, 'start': 0.24, 'end': 0.82, 'word': 'You'},
 {'conf': 0.999512, 'start': 0.82, 'end': 1.14, 'word': 'are'},
 {'conf': 0.991211, 'start': 1.16, 'end': 1.46, 'word': 'my'},
 {'conf': 0.984863, 'start': 1.46, 'end': 1.82, 'word': 'all'},
 {'conf': 0.953125, 'start': 1.82, 'end': 1.98, 'word': 'the'},
 {'conf': 1.0, 'start': 1.98, 'end': 2.4, 'word': 'world'},
 {'conf': 0.368896, 'start': 2.4, 'end': 3.44, 'word': 'and'},
 {'conf': 0.808105, 'start': 3.44, 'end': 3.66, 'word': 'i'},
 {'conf': 0.999023, 'start': 3.66, 'end': 3.96, 'word': 'must'},
 {'conf': 1.0, 'start': 3.96, 'end': 4.48, 'word': 'strive'},
 {'conf': 0.998535, 'start': 4.48, 'end': 5.24, 'word': 'to'},
 {'conf': 0.999512, 'start': 5.24, 'end': 5.5, 'word': 'know'},
 {'conf': 0.999512, 'start': 5.5, 'end': 5.76, 'word': 'my'},
 {'conf': 0.998047, 'start': 5.76, 'end': 6.42, 'word': 'shames'},
 {'conf': 0.999512, 'start': 6.42, 'end': 6.74, 'word': 'and'},
 {'conf': 0.999756, 'start': 6.74, 'end': 7.26, 'w